# Data Analysis Homework 1: Pandas and Numpy

## Q1) 1. Load transit_segments.csv and vessel_information.csv and print the first 5 rows

In [194]:
import numpy as np
import pandas as pd

In [195]:
transit_data = pd.read_csv("./data/AIS/transit_segments.csv")
print(transit_data.head())

   mmsi               name  transit  segment  seg_length  avg_sog  min_sog  \
0     1        Us Govt Ves        1        1         5.1     13.2      9.2   
1     1  Dredge Capt Frank        1        1        13.5     18.6     10.4   
2     1      Us Gov Vessel        1        1         4.3     16.2     10.3   
3     1      Us Gov Vessel        2        1         9.2     15.4     14.5   
4     1  Dredge Capt Frank        2        1         9.2     15.4     14.6   

   max_sog  pdgt10        st_time       end_time  
0     14.5    96.5  2/10/09 16:03  2/10/09 16:27  
1     20.6   100.0   4/6/09 14:31   4/6/09 15:20  
2     20.5   100.0   4/6/09 14:36   4/6/09 14:55  
3     16.1   100.0  4/10/09 17:58  4/10/09 18:34  
4     16.2   100.0  4/10/09 17:59  4/10/09 18:35  


In [196]:
vessel_data = pd.read_csv("./data/AIS/vessel_information.csv")
print(vessel_data.head())

   mmsi  num_names                                              names sov  \
0     1          8  Bil Holman Dredge/Dredge Capt Frank/Emo/Offsho...   Y   
1     9          3                         000000009/Raven/Shearwater   N   
2    21          1                                      Us Gov Vessel   Y   
3    74          2                                  Mcfaul/Sarah Bell   N   
4   103          3           Ron G/Us Navy Warship 103/Us Warship 103   Y   

      flag flag_type  num_loas                                    loa  \
0  Unknown   Unknown         7  42.0/48.0/57.0/90.0/138.0/154.0/156.0   
1  Unknown   Unknown         2                              50.0/62.0   
2  Unknown   Unknown         1                                  208.0   
3  Unknown   Unknown         1                                  155.0   
4  Unknown   Unknown         2                             26.0/155.0   

   max_loa  num_types                             type  
0    156.0          4  Dredging/MilOps/Re

## Q1) 2. Keep only those rows with the type value occurring for at least 99 times

In [197]:
type_counts = vessel_data['type'].value_counts()
print("Total Number of Types : ",type_counts.count())
type_counts = type_counts[type_counts >= 99]
print("Total Number of Types Occuring for atleast 99 times : ",type_counts.count())
print(type_counts)

Total Number of Types :  206
Total Number of Types Occuring for atleast 99 times :  10
type
Cargo        5622
Tanker       2440
Pleasure      601
Tug           221
Sailing       205
Fishing       200
Other         178
Passenger     150
Towing        117
Unknown       106
Name: count, dtype: int64


In [198]:
print("Total Number of Records before dropping : ",vessel_data.shape[0])
vessel_data = vessel_data[vessel_data['type'].isin(type_counts.index)]
print("Total Number of Records After dropping : ",vessel_data.shape[0])

Total Number of Records before dropping :  10771
Total Number of Records After dropping :  9840


## Q1) 3. Merge data/AIS/vessel_information.csv and data/AIS/transit_segments.csv on the "mmsi" column using outer join

In [199]:
merged_data_outer = pd.merge(vessel_data,transit_data,on="mmsi",how="outer")
print(merged_data_outer.columns)
print("Total Number of Records after outer join : ",merged_data_outer.shape[0])

Index(['mmsi', 'num_names', 'names', 'sov', 'flag', 'flag_type', 'num_loas',
       'loa', 'max_loa', 'num_types', 'type', 'name', 'transit', 'segment',
       'seg_length', 'avg_sog', 'min_sog', 'max_sog', 'pdgt10', 'st_time',
       'end_time'],
      dtype='object')
Total Number of Records after outer join :  262526


## Q1) 4. Derive the inner join results without directly calling inner join in pandas

#### Analysis : Outer join retains all the records even if they are not present in the other dataset. Hence, the merged dataset will have 'NaN' to all the columns of a transit dataset whose records are not present in vessel dataset and vice versa. Inner join contains the records whose mmsi is present in both the dataset. Hence, the merged dataset doesnt have 'NaN'. We can use this and remove all the records whose columns has NaN. The result will be the inner join of the two datasets

In [200]:
merged_data_inner = merged_data[
    merged_data['type'].notna() &   # column from vessel_information
    merged_data['max_sog'].notna()  # column from transit_segments
]
print(merged_data_inner.columns)
print("Total Number of Records in inner join result without using inner join function : ",merged_data_inner.shape[0])

Index(['mmsi', 'num_names', 'names', 'sov', 'flag', 'flag_type', 'num_loas',
       'loa', 'max_loa', 'num_types', 'type', 'name', 'transit', 'segment',
       'seg_length', 'avg_sog', 'min_sog', 'max_sog', 'pdgt10', 'st_time',
       'end_time'],
      dtype='object')
Total Number of Records in inner join result without using inner join function :  197814


## Q1) 5. Directly call the inner join provided by Pandas, check whether your results above are exactly the same

#### Analysis : The result is same for the approaches. Meaning that the row count matched for both the approaches. This suggests that we can remove the NaN rows from outer join and produce inner join output.

In [201]:
inner_join_result = pd.merge(vessel_data,transit_data,on="mmsi",how="inner")
print(inner_join_result.columns)
print("Total Number of Records in inner join result using inner join function : ",inner_join_result.shape[0])

Index(['mmsi', 'num_names', 'names', 'sov', 'flag', 'flag_type', 'num_loas',
       'loa', 'max_loa', 'num_types', 'type', 'name', 'transit', 'segment',
       'seg_length', 'avg_sog', 'min_sog', 'max_sog', 'pdgt10', 'st_time',
       'end_time'],
      dtype='object')
Total Number of Records in inner join result using inner join function :  197814


## Q1) 6. Save merged dataset as AIS_merge.csv and check the missing values. Replace missing values with column mode.

In [202]:
merged_data_outer.to_csv("./AIS_merge.csv",index=False)

In [203]:
AIS_merge_data = pd.read_csv("./AIS_merge.csv")
print(AIS_merge_data.columns)

Index(['mmsi', 'num_names', 'names', 'sov', 'flag', 'flag_type', 'num_loas',
       'loa', 'max_loa', 'num_types', 'type', 'name', 'transit', 'segment',
       'seg_length', 'avg_sog', 'min_sog', 'max_sog', 'pdgt10', 'st_time',
       'end_time'],
      dtype='object')


In [204]:
AIS_merge_data.isnull().sum()

mmsi              0
num_names     64712
names         64712
sov           64712
flag          64712
flag_type     64712
num_loas      64712
loa           64712
max_loa       64712
num_types     64712
type          64712
name              0
transit           0
segment           0
seg_length        0
avg_sog           0
min_sog           0
max_sog           0
pdgt10            0
st_time           0
end_time          0
dtype: int64

In [210]:
print("Mode of the all columns")
print(AIS_merge_data.mode().iloc[0])
AIS_merge_data = AIS_merge_data.fillna(AIS_merge_data.mode().iloc[0])

Mode of the all columns
mmsi                          367063030
num_names                           1.0
names               Emerald Princess Ii
sov                                   N
flag          United States of America 
flag_type                       Foreign
num_loas                            1.0
loa                               294.0
max_loa                           294.0
num_types                           1.0
type                              Cargo
name                Emerald Princess Ii
transit                               1
segment                               1
seg_length                         19.9
avg_sog                             9.7
min_sog                             0.0
max_sog                            10.0
pdgt10                              0.0
st_time                   7/19/09 16:55
end_time                   4/19/09 8:06
Name: 0, dtype: object


In [211]:
AIS_merge_data.isnull().sum()

mmsi          0
num_names     0
names         0
sov           0
flag          0
flag_type     0
num_loas      0
loa           0
max_loa       0
num_types     0
type          0
name          0
transit       0
segment       0
seg_length    0
avg_sog       0
min_sog       0
max_sog       0
pdgt10        0
st_time       0
end_time      0
dtype: int64

## Q1) 7. Use z-scores to detect outliers

In [212]:
print(AIS_merge_data.dtypes)

mmsi            int64
num_names     float64
names          object
sov            object
flag           object
flag_type      object
num_loas      float64
loa            object
max_loa       float64
num_types     float64
type           object
name           object
transit         int64
segment         int64
seg_length    float64
avg_sog       float64
min_sog       float64
max_sog       float64
pdgt10        float64
st_time        object
end_time       object
dtype: object


In [213]:
AIS_merge_data["z_mmsi"] = (AIS_merge_data["mmsi"] - AIS_merge_data["mmsi"].mean()) /AIS_merge_data["mmsi"].std()
outliers_mmsi = AIS_merge_data[AIS_merge_data["z_mmsi"]>3]
print(len(outliers_mmsi))

386


In [214]:
AIS_merge_data["z_num_names"] = (AIS_merge_data["num_names"] - AIS_merge_data["num_names"].mean()) /AIS_merge_data["num_names"].std()
outliers_num_names = AIS_merge_data[AIS_merge_data["z_num_names"]>3]
print(len(outliers_num_names))

405


In [215]:
AIS_merge_data["z_num_loas"] = (AIS_merge_data["num_loas"] - AIS_merge_data["num_loas"].mean()) /AIS_merge_data["num_loas"].std()
outliers_num_loas = AIS_merge_data[AIS_merge_data["z_num_loas"]>3]
print(len(outliers_num_loas))

9970


In [216]:
AIS_merge_data["z_max_loa"] = (AIS_merge_data["max_loa"] - AIS_merge_data["max_loa"].mean()) /AIS_merge_data["max_loa"].std()
outliers_max_loa = AIS_merge_data[AIS_merge_data["z_max_loa"]>3]
print(len(outliers_max_loa))

941


In [217]:
AIS_merge_data["z_num_types"] = (AIS_merge_data["num_types"] - AIS_merge_data["num_types"].mean()) /AIS_merge_data["num_types"].std()
outliers_num_types = AIS_merge_data[AIS_merge_data["z_num_types"]>3]
print(len(outliers_num_types))

0


In [218]:
AIS_merge_data["z_transit"] = (AIS_merge_data["transit"] - AIS_merge_data["transit"].mean()) / AIS_merge_data["transit"].std()
outliers_transit = AIS_merge_data[np.abs(AIS_merge_data["z_transit"]) > 3]
print(len(outliers_transit))

4790


In [219]:
AIS_merge_data["z_segment"] = (AIS_merge_data["segment"] - AIS_merge_data["segment"].mean()) / AIS_merge_data["segment"].std()
outliers_segment = AIS_merge_data[np.abs(AIS_merge_data["z_segment"]) > 3]
print(len(outliers_segment))

365


In [220]:
AIS_merge_data["z_seg_length"] = (AIS_merge_data["seg_length"] - AIS_merge_data["seg_length"].mean()) / AIS_merge_data["seg_length"].std()
outliers_seg_length = AIS_merge_data[np.abs(AIS_merge_data["z_seg_length"]) > 3]
print(len(outliers_seg_length))

1330


In [221]:
AIS_merge_data["z_avg_sog"] = (AIS_merge_data["avg_sog"] - AIS_merge_data["avg_sog"].mean()) / AIS_merge_data["avg_sog"].std()
outliers_avg_sog = AIS_merge_data[np.abs(AIS_merge_data["z_avg_sog"]) > 3]
print(len(outliers_avg_sog))

2028


In [222]:
AIS_merge_data["z_min_sog"] = (AIS_merge_data["min_sog"] - AIS_merge_data["min_sog"].mean()) / AIS_merge_data["min_sog"].std()
outliers_min_sog = AIS_merge_data[np.abs(AIS_merge_data["z_min_sog"]) > 3]
print(len(outliers_min_sog))

2254


In [223]:
AIS_merge_data["z_max_sog"] = (AIS_merge_data["max_sog"] - AIS_merge_data["max_sog"].mean()) / AIS_merge_data["max_sog"].std()
outliers_max_sog = AIS_merge_data[np.abs(AIS_merge_data["z_max_sog"]) > 3]
print(len(outliers_max_sog))

4480


In [224]:
AIS_merge_data["z_pdgt10"] = (AIS_merge_data["pdgt10"] - AIS_merge_data["pdgt10"].mean()) / AIS_merge_data["pdgt10"].std()
outliers_pdgt10 = AIS_merge_data[np.abs(AIS_merge_data["z_pdgt10"]) > 3]
print(len(outliers_pdgt10))

0


In [225]:
print(AIS_merge_data.shape[0])

262526


In [226]:
print("Total outliers : ",len(outliers_mmsi)+len(outliers_num_names)+len(outliers_num_loas)+len(outliers_max_loa)+len(outliers_num_types)+len(outliers_transit)+len(outliers_segment)+len(outliers_seg_length)+len(outliers_avg_sog)+len(outliers_min_sog)+len(outliers_max_sog)+len(outliers_pdgt10))

Total outliers :  26949


#### Analysis : The number of outliers are very huge, hence removing the outliers is not feasible. We can implement log normalization or minmax normalization to reduce the number of outliers.

## Q2) 1. create array X of shape (4, 3)

In [227]:
np.random.seed(42)
X = np.random.randint(0, 10, size=(4, 3))
print(X)

[[6 3 7]
 [4 6 9]
 [2 6 7]
 [4 3 7]]


## Q2) 2.write a function dist() to measure the Euclidean distance

In [228]:
n = X.shape[0]
for i in range(n):
    for j in range(n):
        point_i = X[i]
        point_j = X[j]
        if i == j:
            print(f"Point {i+1} to Point {j+1}: 0.000 (distance to itself)")
        else:
            diff = point_i - point_j
            squared_diffs = diff ** 2
            distance = np.sqrt(np.sum(squared_diffs))
            print(f"Point {i+1} {point_i} to Point {j+1} {point_j} : {distance:.3f}")

Point 1 to Point 1: 0.000 (distance to itself)
Point 1 [6 3 7] to Point 2 [4 6 9] : 4.123
Point 1 [6 3 7] to Point 3 [2 6 7] : 5.000
Point 1 [6 3 7] to Point 4 [4 3 7] : 2.000
Point 2 [4 6 9] to Point 1 [6 3 7] : 4.123
Point 2 to Point 2: 0.000 (distance to itself)
Point 2 [4 6 9] to Point 3 [2 6 7] : 2.828
Point 2 [4 6 9] to Point 4 [4 3 7] : 3.606
Point 3 [2 6 7] to Point 1 [6 3 7] : 5.000
Point 3 [2 6 7] to Point 2 [4 6 9] : 2.828
Point 3 to Point 3: 0.000 (distance to itself)
Point 3 [2 6 7] to Point 4 [4 3 7] : 3.606
Point 4 [4 3 7] to Point 1 [6 3 7] : 2.000
Point 4 [4 3 7] to Point 2 [4 6 9] : 3.606
Point 4 [4 3 7] to Point 3 [2 6 7] : 3.606
Point 4 to Point 4: 0.000 (distance to itself)


## Q2) 3. Consider adding a new point and add it to X using broadcasting

#### Case 1 : A datapoint with coordinate ( 3, 1, 2). This point can be added because, the matrix X and the new data point has same dimension. Hence, can be broadcasted.

In [229]:
point1 = np.array([3, 1, 2])
print("Dimension of the data point : ", point1.shape)
print("Dimension of the matrix X : ", X.shape[1])

Dimension of the data point :  (3,)
Dimension of the matrix X :  3


In [230]:
print(X)

[[6 3 7]
 [4 6 9]
 [2 6 7]
 [4 3 7]]


In [231]:
result1 = X+point1
print(result1)

[[ 9  4  9]
 [ 7  7 11]
 [ 5  7  9]
 [ 7  4  9]]


#### Case 2 : A datapoint with coordinate (9, 6). This point cannot be added because, the matrix X and the new data point has different dimension. Hence, cannot be broadcasted.

In [233]:
point2 = np.array([9,6])
print("Dimension of the data point : ", point2.shape)
print("Dimension of the matrix X : ", X.shape[1])

Dimension of the data point :  (2,)
Dimension of the matrix X :  3


In [234]:
print(X)

[[6 3 7]
 [4 6 9]
 [2 6 7]
 [4 3 7]]


In [235]:
result2 = X+point2
print(result2)

ValueError: operands could not be broadcast together with shapes (4,3) (2,) 

#### Case 3 : A datapoint with coordinate (2). This point can be added because, even if the matrix X and the new data point has different dimension,the new point is considered as scalar value and will be added to every element in the matrix. Hence, can be broadcasted.

In [236]:
point3 = np.array([2])
print("Dimension of the data point : ", point3.shape)
print("Dimension of the matrix X : ", X.shape[1])

Dimension of the data point :  (1,)
Dimension of the matrix X :  3


In [237]:
print(X)

[[6 3 7]
 [4 6 9]
 [2 6 7]
 [4 3 7]]


In [238]:
result3 = X+point3
print(result3)

[[ 8  5  9]
 [ 6  8 11]
 [ 4  8  9]
 [ 6  5  9]]
